In [15]:
from   ete3                         import Tree, TreeStyle, NodeStyle, faces
from   goatools.anno.idtogos_reader import IdToGosReader
from   goatools.go_enrichment       import GOEnrichmentStudy
from   goatools.obo_parser          import GODag
from   goatools.utils               import read_geneset
from   pathlib import Path
import pandas  as pd
import sys
import re

In [16]:
sp_tree_file  = "species_tree.ultra.nwk"
enc_og_tsv    = "Orthogroups.Encoded.tsv"
og_codes      = "og_codes.tsv"
sp_list_file  = "species_list"
go_terms_file = "GO_master_file.tsv"
sp_list_fh    = open(sp_list_file, "r") 
sp_list_data  = sp_list_fh.read() 
species_list  = sp_list_data.split("\n") 
species_list  = list(filter(None, species_list))
species_count = len(species_list)
root_species  = "Cryptosporidium_parvum"
sp_list_fh.close()
species_tree  = Tree(sp_tree_file)
species_tree.set_outgroup(root_species)
species_tree.convert_to_ultrametric()
species_tree.ladderize()
enc_og_df   = pd.read_csv(enc_og_tsv,sep="\t",dtype=str)
og_codes_df = pd.read_csv(og_codes,  sep="\t",dtype=str)
go_terms_df = pd.read_csv(go_terms_file,sep="\t",dtype=str)

In [17]:
def get_matrix_code(matrix_str,matrix_type):
    matrix_list = list(matrix_str)
    dot_count = 0
    one_count = 0
    for matrix_element in range(len(matrix_list)):
        if matrix_list[matrix_element] == ".":
            dot_count += 1
        if matrix_list[matrix_element] == "1":
            one_count += 1
    loss_tolerance = dot_count - 1
    gain_tolerance = one_count - 1
    if (matrix_type == "gain"):
        tolerance = gain_tolerance
    elif(matrix_type == "loss"):
        tolerance = loss_tolerance
    mat_df              = og_codes_df.copy()
    mat_df["match"]     = mat_df["matrix"].apply(lambda x: re.findall(matrix_str,x)[0] if re.findall(matrix_str,x) else "")
    index_list          = mat_df["match"].apply(lambda x: True if x else False)
    mat_df              = mat_df[index_list]
    mat_df["match_sum"] = mat_df["match"].apply(lambda x: sum(list(map(int,list(x)))))
    mat_df              = mat_df[mat_df["match_sum"]>=tolerance]
    code_list           = mat_df["code"].values.flatten().tolist()
    return code_list

In [18]:
counter   = 0
node_list = []
core_gain = []
core_loss = []
excl_gain = []
excl_loss = []
parent    = []
species   = []

In [19]:
for node in species_tree.traverse():
    counter += 1
    node.add_feature("node_id",counter)
    excl_gain_base_list = ["0"] * species_count
    core_gain_base_list = ["0"] * species_count
    excl_loss_base_list = ["1"] * species_count
    core_loss_base_list = ["."] * species_count
    cur_species_list    = []
    cur_index_list      = []
    if (node.is_leaf() == True):
        cur_species_list = [node.name]
    else:
        for sub_node in node.get_descendants():
            if( sub_node.is_leaf() == True):
                cur_species_list.append(sub_node.name)
    for cur_species in cur_species_list:
        cur_index = species_list.index(cur_species)
        cur_index_list.append(cur_index)
    for cur_index in cur_index_list:
        excl_gain_base_list[cur_index] = "1"
        core_gain_base_list[cur_index] = "."
        excl_loss_base_list[cur_index] = "0"
        core_loss_base_list[cur_index] = "0"
    if (node.is_root()==True):
        parent_id = ""
    else:
        parent_id = node.up.node_id
    core_gain_matrix = ''.join(list_item for list_item in core_gain_base_list)
    excl_gain_matrix = ''.join(list_item for list_item in excl_gain_base_list)
    excl_gain_matrix = excl_gain_matrix[::-1]
    core_loss_matrix = ''.join(list_item for list_item in core_loss_base_list)
    excl_loss_matrix = ''.join(list_item for list_item in excl_loss_base_list)
    excl_loss_matrix = excl_loss_matrix[::-1]
    core_gain_code   = get_matrix_code(core_gain_matrix,"gain")
    excl_gain_code   = int(excl_gain_matrix,base=2)
    core_loss_code   = get_matrix_code(core_loss_matrix,"loss")
    excl_loss_code   = int(excl_loss_matrix,base=2)
    core_gain_index  = enc_og_df["Total"].isin(core_gain_code)
    excl_gain_index  = enc_og_df["Total"] == str(excl_gain_code)
    core_loss_index  = enc_og_df["Total"].isin(core_loss_code)
    excl_loss_index  = enc_og_df["Total"] == str(excl_loss_code)
    core_gain_list   = enc_og_df[core_gain_index]["Orthogroup"].values.flatten().tolist()
    excl_gain_list   = enc_og_df[excl_gain_index]["Orthogroup"].values.flatten().tolist()
    core_loss_list   = enc_og_df[core_loss_index]["Orthogroup"].values.flatten().tolist()
    excl_loss_list   = enc_og_df[excl_loss_index]["Orthogroup"].values.flatten().tolist()
    node_list.append(counter)
    core_gain.append(core_gain_list)
    core_loss.append(core_loss_list)
    excl_gain.append(excl_gain_list)
    excl_loss.append(excl_loss_list)
    parent.append(parent_id)
    species.append(cur_species_list)

In [20]:
gain_dict = {"node"         : node_list,
             "core_gain"    : core_gain,
             "core_loss"    : core_loss,
             "excl_gain"    : excl_gain,
             "excl_loss"    : excl_loss,
             "species_list" : species,
             "parent"       : parent}
gain_df = pd.DataFrame(gain_dict)

In [21]:
def get_parent_ogs(node_id):
    parent_id      = gain_df[gain_df["node"]==node_id]["parent"].values.flatten().tolist()[0]
    if (parent_id!=""):
        parent_og_list = gain_df[gain_df["node"]==parent_id]["core_gain"].values.flatten().tolist()[0]
    else:
        parent_og_list = gain_df[gain_df["node"]==node_id]["core_gain"].values.flatten().tolist()[0]
    return parent_og_list
def get_parent_sps(node_id):
    parent_id      = gain_df[gain_df["node"]==node_id]["parent"].values.flatten().tolist()[0]
    if (parent_id!=""):
        parent_sp_list = gain_df[gain_df["node"]==parent_id]["species_list"].values.flatten().tolist()[0]
    else:
        parent_sp_list = gain_df[gain_df["node"]==node_id]["species_list"].values.flatten().tolist()[0]
    return parent_sp_list

In [22]:
gain_df["parent_og_list"] = gain_df["node"].apply(lambda x: get_parent_ogs(x))
gain_df["parent_sp_list"] = gain_df["node"].apply(lambda x: get_parent_sps(x))

In [23]:
def run_node_enrichment(node,event_type):
    node_og_list   = gain_df[gain_df["node"]==node][event_type].values.flatten().tolist()[0]
    if(len(node_og_list)>0):
        node_sp_list   = gain_df[gain_df["node"]==node]["species_list"].values.flatten().tolist()[0]
        parent_og_list = gain_df[gain_df["node"]==node]["parent_og_list"].values.flatten().tolist()[0]
        parent_sp_list = gain_df[gain_df["node"]==node]["parent_sp_list"].values.flatten().tolist()[0]
        parent_go_df   = go_terms_df.copy()
        parent_go_df   = parent_go_df[(parent_go_df["species"].isin(parent_sp_list))&(parent_go_df["orthogroup"].isin(parent_og_list))]
        parent_go_ogs  = list(set(parent_go_df["orthogroup"].values.flatten().tolist()))
        parent_go_gos  = []
        for parent_go_og in parent_go_ogs:
            og_go_list = list(set(parent_go_df[parent_go_df["orthogroup"]==parent_go_og]["go_id"].values.flatten().tolist()))
            og_go_list = [ "GO:"+ og_go for og_go in og_go_list]
            og_go_str  = ";".join(og_go_list)
            parent_go_gos.append(og_go_str)
        parent_go_og_dict = {"orthogroup" : parent_go_ogs, "go_terms" : parent_go_gos}
        parent_go_og_df   = pd.DataFrame(parent_go_og_dict)
        parent_go_og_df.to_csv("bg.go.tsv",sep="\t",header=None,index=False)
        with open("bg_ogs.txt",'w') as fh:
            for parent_og in parent_og_list:
                fh.write("%s\n" % parent_og)
        with open("sel_ogs.txt",'w') as fh:
            for node_og in node_og_list:
                fh.write("%s\n" % node_og)
        sel_ids   = read_geneset("sel_ogs.txt")
        bg_ids    = read_geneset("bg_ogs.txt")
        godag     = GODag("go.obo")
        annoobj   = IdToGosReader("bg.go.tsv",godag=godag)
        goeaobj   = GOEnrichmentStudy(bg_ids,annoobj.get_id2gos(),godag,methods=['bonferroni','fdr_bh'],pvalcalc='fisher_scipy_stats')
        goea_res  = goeaobj.run_study_nts(sel_ids)
        if (len(goea_res)>0):
            goea_cols = ["GO","name","NS","enrichment","p_uncorrected","p_fdr_bh","p_bonferroni","study_count","study_n","pop_count","pop_n","level","study_items","pop_items"]
            goea_df   = pd.DataFrame(goea_res)
            goea_df   = goea_df[goea_df["p_bonferroni"]<0.05]
            goea_df   = goea_df[goea_cols]
            return goea_df
        else:
            return False
    else:
        return False  

In [26]:
def filter_goea_df(goea_df,NS):
    if(type(goea_df)!=bool):
        tmp_df = goea_df.copy()
        tmp_df = tmp_df[tmp_df["NS"]==NS]
        levels = list(set(tmp_df["level"].values.flatten().tolist()))
        levels = levels[-2:]
        tmp_df = tmp_df[tmp_df["level"].isin(levels)]
        return tmp_df
    else:
        return False

In [ ]:
NSs  = ["BP","MF","CC"]
event_types = ["core_gain","excl_gain","core_loss","excl_loss"]
for node in species_tree.traverse():
    if (node.is_root()==False):
        for event_type in event_types:
            event_df = run_node_enrichment(node.node_id,event_type)
            for NS in NSs:
                tsv_file = str(node.node_id)+"."+event_type+"."+NS+".tsv"
                cur_df   = filter_goea_df(event_df,NS)
                if(type(cur_df)!=bool):
                    cur_df.to_csv(tsv_file,sep="\t",index=False)

In [29]:
node = 28
NSs  = ["BP","MF","CC"]
event_types = ["excl_gain"]
for event_type in event_types:
    event_df = run_node_enrichment(node,event_type)
    for NS in NSs:
        tsv_file = str(node)+"."+event_type+"."+NS+".tsv"
        cur_df   = filter_goea_df(event_df,NS)
        if(type(cur_df)!=bool):
            cur_df.to_csv(tsv_file,sep="\t",index=False)

      316 READ: sel_ogs.txt
      622 READ: bg_ogs.txt
go.obo: fmt(1.2) rel(2023-05-10) 46,490 Terms
HMS:0:00:00.055836   1,125 annotations READ: bg.go.tsv 
390 IDs in loaded association branch, biological_process

Load  Ontology Enrichment Analysis ...
Propagating term counts up: is_a
 63%    390 of    622 population items found in association

Runing  Ontology Analysis: current study set of 316 IDs.
 46%    144 of    316 study items found in association
100%    316 of    316 study items found in population(622)
Calculating 678 uncorrected p-values using fisher_scipy_stats
     678 terms are associated with    390 of    622 population items
     434 terms are associated with    144 of    316 study items
  METHOD bonferroni:
      26 GO terms found significant (< 0.05=alpha) (  5 enriched +  21 purified): local bonferroni
      54 study items associated with significant GO IDs (enriched)
     105 study items associated with significant GO IDs (purified)
  METHOD fdr_bh:
      46 GO ter

In [30]:
event_df

,GO,name,NS,enrichment,p_uncorrected,p_fdr_bh,p_bonferroni,study_count,study_n,pop_count,pop_n,level,study_items,pop_items
0,GO:0044238,primary metabolic process,BP,e,4.271697e-05,1.207586e-03,2.896211e-02,34,316,42,622,2,"{OG0028541, OG0028558, OG0028446, OG0028510, O...","{OG0028541, OG0028558, OG0028446, OG0028510, O..."
1,GO:0006807,nitrogen compound metabolic process,BP,e,4.538087e-05,1.230729e-03,3.076823e-02,32,316,39,622,2,"{OG0028541, OG0028558, OG0028446, OG0028510, O...","{OG0028541, OG0028558, OG0028446, OG0028510, O..."
229,GO:0003824,catalytic activity,MF,e,2.848000e-07,1.206840e-05,1.930944e-04,43,316,51,622,1,"{OG0028541, OG0028436, OG0028446, OG0028512, O...","{OG0006569, OG0028541, OG0028436, OG0028446, O..."
230,GO:0016740,transferase activity,MF,e,4.274640e-05,1.207586e-03,2.898206e-02,26,316,30,622,2,"{OG0028541, OG0028436, OG0028446, OG0028512, O...","{OG0028541, OG0028436, OG0028446, OG0029066, O..."
231,GO:0016772,"transferase activity, transferring phosphorus-...",MF,e,6.311212e-05,1.645770e-03,4.279002e-02,18,316,19,622,3,"{OG0028470, OG0028436, OG0028576, OG0028479, O...","{OG0028436, OG0028446, OG0029066, OG0028442, O..."
515,GO:0018995,host cellular component,CC,p,2.963363e-58,2.009160e-55,2.009160e-55,0,316,151,622,2,{},"{OG0006534, OG0028413, OG0029030, OG0006597, O..."
516,GO:0110165,cellular anatomical entity,CC,p,5.371590e-34,1.820969e-31,3.641938e-31,97,316,337,622,1,"{OG0028558, OG0028545, OG0028450, OG0028567, O...","{OG0028413, OG0029030, OG0006630, OG0029034, O..."
517,GO:0005575,cellular_component,CC,p,3.847167e-33,8.491102e-31,2.608379e-30,99,316,339,622,0,"{OG0028558, OG0028545, OG0028450, OG0028523, O...","{OG0028413, OG0029030, OG0006630, OG0029034, O..."
518,GO:0043657,host cell,CC,p,5.009500e-33,8.491102e-31,3.396441e-30,0,316,93,622,3,{},"{OG0006534, OG0029030, OG0006597, OG0005473, O..."
519,GO:0033643,host cell part,CC,p,1.853693e-31,2.513608e-29,1.256804e-28,0,316,89,622,3,{},"{OG0028413, OG0006597, OG0005473, OG0006539, O..."
